# Key Figures Electrical Installations >36kV
Notebook to get key figures of the geodataset Electrical installations with a nominal voltage exceeding 36 kV

In [ ]:
pip install geopandas

In [2]:
import geopandas as gpd

In [8]:
url = "https://data.geo.admin.ch/ch.bfe.elektrische-anlagen_ueber_36/gpkg/2056/ElektrischeAnlagenNennspannungUeber36kV_V1.gpkg"
dfLeitung = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Leitung')

In [18]:
dfLeitung["Length"] = dfLeitung.length
dfLength = dfLeitung.groupby(['Spannung'])['Length'].sum()

In [16]:
dfLeitung[dfLeitung['Spannung'] =='Andere'].groupby(['StromnetzTyp'])['Length'].sum()

StromnetzTyp
Allgemeinstrom    3.807751e+06
Name: Length, dtype: float64

In [23]:
dfLength.to_frame().transpose()

Spannung,Andere,S110kV,S132kV,S150kV,S220kV,S380kV,S50kV,S60kV,S65kV
Length,3.807751e+06,1.792030e+06,96249.917978,460775.593508,4.857297e+06,1.915187e+06,1.045459e+06,46456.976728,15614.107936
